In [0]:
#imports 

from pyspark.sql.functions import explode, col, arrays_zip, month, when, round, current_timestamp
from utils.transform import explode_struct

In [0]:
#Variaveis do Unity Catalog
CATALOGO = "agroclima"
SCHEMA = "silver"
TABELA = "temperaturas"

# Montar caminho da tabela
caminho_tabela = f"{CATALOGO}.{SCHEMA}.{TABELA}"

In [0]:
# Carrega Tabela de Temperaturas
df_temperatura = spark.read.table("agroclima.bronze.temperatura")

#### Transformação

In [0]:
# junta os elementos correpondentes de cada array em uma so estrutura
df_transformado = df_temperatura.select(
                            "*",
                            arrays_zip(
                                col("daily.time"),
                                col("daily.temperature_2m_max"),
                                col("daily.temperature_2m_min"),
                                col("daily.relative_humidity_2m_max"),
                                col("daily.relative_humidity_2m_min"),
                                col("daily.sunshine_duration"),
                                col("daily.precipitation_hours"),
                                col("daily.daylight_duration"),
                                col("daily.precipitation_sum"),
                                col('daily.rain_sum')
                            ).alias('valores_zipados')
                    )

In [0]:
#Explode para que cada conjunto fique em uma linha
df_transformado = df_transformado.select(
                            "*",
                            explode(col("valores_zipados")).alias("valores")
).drop("valores_zipados")

In [0]:
#Transforma os valores em colunas separadas
df_transformado = df_transformado.select(
                            "*",
                            col("valores.time").alias("data")
                        )

In [0]:
#Explode Estrutura em Linhas
df_transformado = explode_struct(df_transformado, "valores", "tipo", "valor")

In [0]:
# Transforma a coluna de data em Date
df_transformado = df_transformado.withColumn("data", col("data").cast("date"))

In [0]:
#Cria Coluna de Mês para particionamento dos dados
df_transformado = df_transformado.withColumn("mes", month(col("data")))

#### Limpeza

In [0]:
df_transformado.columns

In [0]:
#Seleciona as colunas necessárias
df_limpeza = df_transformado.select(
                    "municipio_id",
                    "tipo",
                    "valor",
                    "data",
                    "uf",
                    "ano",
                    "mes"

)

In [0]:
# Remove Linhas com o tipo time
df_limpeza = df_limpeza.filter(
        col("tipo") != "time"
)

In [0]:
# Transforma a coluna de valor em double e ano em int
df_limpeza = df_limpeza.withColumn("valor", col("valor").cast("double")) \
                        .withColumn("ano", col("ano").cast("int"))

In [0]:
#Transforma os valores de Duração do Sol e Duração da Luz do Dia de segundos para Horas
df_limpeza = df_limpeza.withColumn(
        "valor",
        when(col("tipo").isin("sunshine_duration", "daylight_duration"), round(col("valor")/3600, 1))
        .otherwise(col("valor"))
)

#display(df_limpeza)

#### Qualidade dos Dados

In [0]:
# Verificando se há linhas duplicadas 
#print(df_limpeza.count())
#print(df_limpeza.distinct().count())

In [0]:
# Verificando Percentual de Valores Nulos por coluna
#df_limpeza.toPandas().isnull().sum() / df_limpeza.count()

In [0]:
# verificando quantidade de linhas e range das colunas 
#df_limpeza.summary().show()

#### Carga

In [0]:
df_carga = df_limpeza.withColumn("_data_ingestao", current_timestamp())

In [0]:
df_carga.write.format("delta") \
              .mode("overwrite") \
              .partitionBy("uf", "tipo", "ano", "mes") \
              .option("overwriteSchema", "true") \
              .saveAsTable(caminho_tabela)